In [1]:
clean_up = True
%run stdPackages.ipynb
ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.
The file _gams_py_gdb2.gdx is still active and was not deleted.
The file _gams_py_gdb3.gdx is still active and was not deleted.
The file _gams_py_gdb4.gdx is still active and was not deleted.


# Household

## 1. Data/settings

Load data, settings:

In [2]:
t = 2018
name = 'm'
module = name+'_HH' # name of module
db_IO = GpyDB(pickle_path = os.path.join(d['data'], f'{name}_{t}')) # load IO database named IO_name
with open(f"{d['data']}\\glob_{t}","rb") as file: # load global settings anmed glob_name
    glob=pickle.load(file)

## 2. Initial values

Set up nesting tree using the nesting structure:

In [3]:
tree = nestingTree.tree(name = f'{module}_ces', tree = db_IO.get('nestHH').to_list()) # individual tree
Tree = nestingTree.aggTree(name = module, trees = {tree.name: tree})(namespace = {str(n)+'_input': n for n in db_IO.get('n')})

*This should be adjusted when we get regulation data. For now:*
* VAT data is included in the IO data, so this part is ok.
* We set the labor tax rate to 0.47 (roughly the average for 2018 data). **This should be adjusted**.
* We set the level of lump sum transfers at 40\% of labor income (roughly the case for 2018 data). **This should be adjusted.**
* We adjust the TotalTax from the households as well:

In [4]:
transferRate = 0.4004
labortaxRate = 0.478

In [5]:
gpyDB.add_or_merge_vals(db_IO, pd.Series(labortaxRate, index = pd.MultiIndex.from_tuples([('HH','L')], names = ['s','n'])), 'tauS') # add labor tax
gpyDB.add_or_merge_vals(db_IO, pd.Series(db_IO.get('vTax').xs(('HH','VAT'))/(db_IO.get('qD').xs('HH').sum()), index = adj.rc_pd(db_IO.get('qD'), Tree.get('s')).index, name = 'tauD'), 'tauD') # add VAT on the demand side
gpyDB.add_or_merge_vals(db_IO, - db_IO.get('qS').xs('L',level='n') * transferRate, 'tauLump')

Adjust the ```TotalTax``` from the household sector based on this new way:

In [6]:
adhocTaxes = (labortaxRate - transferRate)*db_IO.get('qS').xs('L',level='n')
gpyDB.add_or_merge_vals(db_IO, adj.rc_pd(db_IO.get('vTax'), db_IO.get('s_HH')).xs('VAT',level='taxTypes')+adhocTaxes, 'TotalTax')

Export adjusted IO data again:

In [7]:
db_IO.export(repo=d['data'])

Add time index to the IO data:

In [8]:
def addT(symbol, t):
    return adjMultiIndex.bc(symbol, t).reorder_levels(['t']+symbol.index.names if 't' not in symbol.index.names else symbol.index.names)
[db_IO.__setitem__(k, addT(db_IO.get(k), glob.db['t'].vals)) for k in db_IO.getTypes(['variable','scalar_variable'])];

Make a copy for later:

In [9]:
db_IO0 = db_IO.copy() # we are going to adjust data along the way; this keeps a copy of the original data

## 3. Run module

Initialize production module, without any durables at first:

In [10]:
H = mHouseholds.staticConsumer(tree = Tree, glob = glob, kwargs = {'L2C': db_IO.get('L2C')})
aggregateDB.subset_db(db_IO, H.get('s')) # goes through all symbols in db_IO and only keep the elements that are in the set 's' from Tree.db
aggregateDB.subset_db(db_IO, H.get('n')) # goes through all symbols in db_IO and only keep the elements that are in the set 'n' from Tree.db
# robust.robust_merge_dbs(H.s.db, db_IO, priority = 'second')

Add value shares:

*Value shares for nesting part:*

In [11]:
v = valueShares.SimpleRamsey(Tree, db_IO.copy())
v.compile() # set up model structure, and make sure to initialize symbols if they are not yet defined in the database 
v.write(); # write GAMS code used for the model
m = v.run(exportTo = d['work'],ws=ws) # solve the "model".

Use value shares to initialize variables:
* Outputs and inputs are provided by IO data.
* For intermediate goods, assume a price of 1 (default option in the class, so we don't have to do anything) and set value share = quantity.
* Set share parameters to the ones identified by value share system.

In [12]:
gpyDB.GpyDBs_AOM_Second(H.s.db, gpy(adj.rc_pd(m.out_db.get('vD'), H.get('int')).rename('qD'))) # set intermediate goods levels
gpyDB.GpyDBs_AOM_Second(H.s.db, gpy(m.out_db.get('mu').xs(H.get('t0')[0]).rename('mu'))) # set share parameters

Add the frisch parameter as a target in the IO database if we want to have an intensive margin labor elasticity in there:

In [13]:
db_IO['frisch'] = pd.Series(0.25, index = H.get('labor'), name = 'frisch')

Use sneaky calibration method to solve and calibrate:

In [14]:
H.compile(initDB=True)
H.write();
mCalib, cp = H.sneakyCalib(db_IO, ws = ws, loop_kwargs = {'n': 25}, **{'cns': 'CONOPT4'})

### 4. Export 

*Use data from the calibration step in the main database. The ```sneakyCalib``` method defines some new, auxiliary symbols - these are not used.*

In [15]:
[gpyDB.GpyDBs_AOM_Second(H.s.db, mCalib.out_db[k]) for k in H.s.db.symbols];
H.s.setstate('B')

Export:

In [16]:
with open(os.path.join(d['data'], H.name), "wb") as file:
    pickle.dump(H,file)